# Iteration

In [22]:
from collections.abc import Iterable

## 1. Builtin data iterable
- list
- tuple
- dict
- set
- str

In [6]:
cities = [ "Toulouse", "Lyon", "Pau", "Paris" ]
city_t = ("Toulouse", 470000, "31000")
city_dict = {
    "name": "Toulouse",
    "population": 470000,
    "cp": "31000"
}

In [4]:
for city in cities:
    print(city)

Toulouse
Lyon
Pau
Paris


In [5]:
for info in city_t:
    print(info)

Toulouse
470000
31000


In [8]:
# default iteration on dictionnaries with the keys
for info_name in city_dict:
    print(info_name)

name
population
cp


In [9]:
for info_name in city_dict.keys():
    print(info_name)

name
population
cp


In [10]:
for info_value in city_dict.values():
    print(info_value)

Toulouse
470000
31000


In [11]:
for info_name, info_value in city_dict.items():
    print(info_name, "->", info_value)

name -> Toulouse
population -> 470000
cp -> 31000


In [12]:
city = cities[0]
for letter in city:
    print(letter)

T
o
u
l
o
u
s
e


In [13]:
type(cities)

list

In [15]:
# # TypeError: 'int' object is not iterable
# for x in 123:
#     pass

In [16]:
cities.__iter__

<method-wrapper '__iter__' of list object at 0x00000251D6CEEF80>

In [25]:
# duck typing: presence of method __iter__ 
for data in cities, city_t, city_dict, city:
    isIterable1 = '__iter__' in dir(data)
    isIterable2 = isinstance(data, Iterable)
    print(
        f"data: {data}", 
        f"type: {type(data)}", 
        f"iterable (duck typing): {isIterable1}",
        f"iterable (inheritance): {isIterable2}",
        sep="\n\t- "
    ) 

data: ['Toulouse', 'Lyon', 'Pau', 'Paris']
	- type: <class 'list'>
	- iterable (duck typing): True
	- iterable (inheritance): True
data: ('Toulouse', 470000, '31000')
	- type: <class 'tuple'>
	- iterable (duck typing): True
	- iterable (inheritance): True
data: {'name': 'Toulouse', 'population': 470000, 'cp': '31000'}
	- type: <class 'dict'>
	- iterable (duck typing): True
	- iterable (inheritance): True
data: Toulouse
	- type: <class 'str'>
	- iterable (duck typing): True
	- iterable (inheritance): True
